In [10]:
import numpy as np
import scipy as sp
import os
import glob
import shutil
import csv
from PIL import Image
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
%matplotlib inline

In [11]:
def remove_noice(songM):
    breakpoint = 185
    songM[songM > breakpoint] = 255
    songM[songM <= breakpoint] = 0
    return songM

def find_staves(songM):
    def project_to_left(matrix):
        vector = np.sum(matrix,axis=1)
        return vector
    
    def norm_vec(vec):
        vec = vec-min(vec)
        if max(vec) == 0:
            return vec
        return vec/float(max(vec))
        
    def smooth(vec,zahl):
        if zahl == 0:
            return vec
        vec = np.append([(2*vec[0] + vec[1])/3], [(vec[:-2] + 2*vec[1:-1] + vec[2:])/4])
        vec = np.append(vec,[(2*vec[-1] + vec[-2])/3])
        vec_smooth = smooth(vec,zahl-1)
        return vec_smooth
        
    def find_minima(vec):
        indices_minima = argrelextrema(vec, np.greater)
        return indices_minima
    
    def get_corepart(vec, index_left, index_right):
        corepart_number = 0.99
        part_vec = vec[index_left:index_right]
        index_left = np.where(part_vec < corepart_number*part_vec[0])[0][0] + index_left
        index_right = index_right - len(part_vec) + np.where(part_vec < corepart_number*part_vec[-1])[0][-1]
        return index_left, index_right     
        
    def check_if_stave(part):
        vec = project_to_left(part)
        vec = norm_vec(vec) - 0.4
        num = 0
        for zahl in range(len(vec)-1):
            if vec[zahl] >= 0 and vec[zahl+1] < 0:
                num = num + 1
            if vec[zahl] <= 0 and vec[zahl+1] > 0:
                num = num + 1
        return num == 10
        
    def separate_to_staves(matrix, vec, indices_minima):
        index_left = indices_minima[0]
        for zahl,index_right in enumerate(indices_minima[1:]):
            index_left, index_right = get_corepart(vec, index_left, index_right)
            part = matrix[index_left:index_right,:]
            if check_if_stave(part):
                num_files = len(glob.glob('lines/*.jpg'))
                sp.misc.imsave('lines/line%02i.jpg' % num_files, part)
            index_left = index_right
    
    tryer = project_to_left(songM)
    tryer = norm_vec(tryer)
    #plt.plot(tryer)
    
    tryer = smooth(tryer,50)
    tryer = norm_vec(tryer)
    #plt.plot(tryer)
    #plt.show()
    
    index_minima = np.append(0,np.asarray(find_minima(tryer))[0])
    separate_to_staves(songM, tryer, index_minima)   

if os.path.exists('lines'):
    shutil.rmtree('lines')
os.makedirs('lines')

for song_file in sorted(glob.glob('songs/*.jpg')):
    song = Image.open(song_file).convert('L')
    songM = np.array(song)
    songM = remove_noice(songM)
    find_staves(songM)

In [12]:
def find_notes(lineM):
    def project_to_bottom(matrix):
        vector = np.sum(matrix,axis=0)
        return vector
    
    def norm_vec(vector):
        vector = vector-min(vector)
        if max(vector) == 0:
            return vector
        return vector/float(max(vector))
        
    def smooth(vec,zahl):
        if zahl == 0:
            return vec
        vec = np.append([(2*vec[0] + vec[1])/3], [(vec[:-2] + 2*vec[1:-1] + vec[2:])/4])
        vec = np.append(vec,[(2*vec[-1] + vec[-2])/3])
        vec_smooth = smooth(vec,zahl-1)
        return vec_smooth
        
    def find_minima(vector):
        indices_minima = argrelextrema(vector, np.greater)
        return indices_minima
    
    def separate_to_notes(matrix, indices_minima):
        index_left = indices_minima[0]
        for zahl,index_right in enumerate(indices_minima[1:]):
            part = matrix[:,index_left:index_right]
            if not os.path.exists('lines/notes'):
                os.makedirs('lines/notes')
            num_files = len(glob.glob('lines/notes/*.jpg'))
            sp.misc.imsave('lines/notes/note%03i.jpg' % num_files, part)
            index_left = index_right
    
    tryer = project_to_bottom(lineM)
    tryer = norm_vec(tryer)
    #plt.plot(tryer)
    
    tryer = smooth(tryer,26)
    tryer = norm_vec(tryer)
    #plt.plot(tryer)
    #plt.show()
    
    index_minima = np.append(0,np.asarray(find_minima(tryer))[0])
    separate_to_notes(lineM,index_minima)
    
    
for line_file in sorted(glob.glob('lines/*.jpg')):
    line = Image.open(line_file)
    lineM = np.array(line)
    find_notes(lineM)

In [13]:
def equalize(note):
    width = 24
    hight = 50
    
    def ensure_filesize(matrix):
        h_plus = 255*np.ones((hight/2,matrix.shape[1]))
        matrix = np.append(h_plus,matrix,axis=0)
        matrix = np.append(matrix,h_plus,axis=0)
        v_plus = 255*np.ones((matrix.shape[0],width/2))
        matrix = np.append(v_plus,matrix,axis=1)
        matrix = np.append(matrix,v_plus,axis=1)
        return matrix
    
    def shrink_to_size(matrix):
        index_low = matrix.shape[0]/2 - hight/2
        index_up = matrix.shape[0]/2 + hight/2
        index_left = matrix.shape[1]/2 - width/2
        index_right = matrix.shape[1]/2 + width/2        
        return matrix[index_low:index_up,index_left:index_right]
        
    note = ensure_filesize(note)
    note = shrink_to_size(note)
    return note
    

with open('X_notes.csv', 'w') as f:
    writer = csv.writer(f)
    for note_file in sorted(glob.glob('lines/notes/*.jpg')):
        note = Image.open(note_file)
        note = np.asarray(note)
        note = equalize(note)
        to_save = np.reshape(note,(1,note.shape[0]*note.shape[1]))
        writer.writerows(to_save)
        sp.misc.imsave(note_file, note)

In [14]:
is_note = [0,0,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,0,1,  #000 - 019
    1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,0,1,1,1,1,  #020 - 039
    1,1,1,0,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,  #040 - 059
    1,0,1,1,1,1,1,1,0,0,0,0,1,1,1,1,0,1,1,1,  #060 - 079
    0,1,1,1,1,0,1,1,1,0,0,0,1,1,0,1,1,0,1,1,  #080 - 099
    1,0,1,1,1,1,0,1,0,0,0,0,0,0,1,1,1,1,0,1,  #100 - 119
    1,1,0,0,1,1,0,1,1,0,1,1,1,0,0,0,0,1,1,1,  #120 - 139
    1,0,1,1,1,0,0,0,1,1,1,1,0,1,0,0,0,0,0,0,  #140 - 159
    0,0,1,1,1,1,0,1,1,1,0,1,1,1,1,0,1,1,0,0,  #160 - 179
    0,1,1,1,1,0,1,1,1,1,0,1,1,1,1,0,1,1,1,1,  #180 - 199
    0,0,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,0,  #200 - 219
    0,0,1,1,0,1,1,1,0,1,1,1,0,0,0,1,1,0,1,1,  #220 - 239
    1,1,0,1,1,0,1,1,1,1,0,1,1,0,1,1,1,1,0,1,  #240 - 259
    1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,0,1,0,1,1,  #260 - 279
    1,1,0,1,1,1,0,1,0,1,1,1,1,0,1,1,1,0,1,0,  #280 - 299
    1,1,1,1,0,1,1,1,0,1,0,1,1,1,1,0,1,1,1,0,  #300 - 319
    1,0,1,1,1,1,0,1,1,0,1,0,1,1,1,1,1,0,1,1,  #320 - 339
    1,0,1,1,0,1,1,1,0,1,1,0,1,1,1,0,1,1,0,1,  #340 - 359
    1,1,0,1,1,1,1,0,1,1]    #360 - 369

with open('y_notes.csv', 'w') as f:
    writer = csv.writer(f)
    to_save = np.reshape(np.array(is_note),(len(is_note),1))   
    writer.writerows(to_save)